In [ ]:
import tensorflow as tf
import os
from datetime import datetime

def test_s3_read_singlemachine(worker_index, 
                               num_workers, 
                               data_dir, 
                               num_readers,
                               shuffle_buffer_size, 
                               batch_size, 
                               num_prefetch_batches, 
                               num_steps):

    os.environ["S3_REGION"] = 'us-east-1' # region of the dataset
    filename_pattern = os.path.join(s3_data_bucket, 'train-*')
    print("Reading training data from files matching glob pattern %s" % filename_pattern)
    d = tf.data.Dataset.list_files(filename_pattern)
    d = d.shard(num_workers, worker_index)
    d = d.repeat()
    d = d.shuffle(shuffle_buffer_size)
    d = d.repeat()
    d = d.interleave(tf.data.TextLineDataset,
                     cycle_length=num_readers, block_length=1)
    d = d.batch(batch_size)
    d = d.prefetch(num_prefetch_batches)
    iterator = d.make_initializable_iterator()
    it_op = iterator.get_next()
    with tf.Session() as sess:
        sess.run(iterator.initializer)
        for i in range(num_steps):
              res = sess.run(it_op)
              if i % 30 == 0:
                    print("Finished step %s, time: %s, result length: %s" % (i, datetime.now().strftime("%H:%M:%S %D UTC"), len(res)))

In [2]:
%%time

s3_data_bucket = "s3://datapalooza/mnist"

test_s3_read_singlemachine(worker_index=0, 
                           num_workers=20, 
                           data_dir=s3_data_bucket, 
                           num_readers=10, 
                           shuffle_buffer_size=32, 
                           batch_size=32, 
                           num_prefetch_batches=1, 
                           num_steps=int(100))

Reading training data from files matching glob pattern s3://datapalooza/mnist/train-*
Finished step 0, time: 05:59:03 02/14/18 UTC, result length: 32
Finished step 30, time: 05:59:03 02/14/18 UTC, result length: 32
Finished step 60, time: 05:59:03 02/14/18 UTC, result length: 32
Finished step 90, time: 05:59:03 02/14/18 UTC, result length: 32
CPU times: user 568 ms, sys: 412 ms, total: 980 ms
Wall time: 13.8 s
